<a href="https://colab.research.google.com/github/irika-katiyar/irika-katiyar-twitter/blob/main/CAISProject.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

installing necessary libraries and mounting drive

In [1]:
import pandas as pd
import numpy as np
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/My Drive/CAIS Project

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/My Drive/CAIS Project


In [2]:
!pip install keras

Reading in the csv file and making it into a data frame + tweet preprocessing

In [58]:
df = pd.read_csv("/content/drive/My Drive/CAIS Project/dataset/data.csv", header=0)
 
EMBEDDINGS_DIR = '/content/drive/My Drive/2021 Fall Curriculum/Lessons/Lesson 6: RNNs/glove.6B.50d.txt'

%run -i load_data.py
tweets, tweets_preprocessed, labels, word_index, embedding_matrix = load_data(df, EMBEDDINGS_DIR)


1: Setting tweets and labels to columns of df
2: tokenizing tweets
3: padding
4: loading word embeddings
5: finding word embeddings


checking embedding matrix for word america

In [ ]:
word = "america"

idx = word_index[word]

embedding = embedding_matrix[idx]

print(word, "has a word index of: ", idx) 
print()
print(word, "has a word embedding of:\n", embedding)

america has a word index of:  1740

america has a word embedding of:
 [-0.13124     0.46555001 -0.10921     0.18759     0.073319   -0.40072
 -1.14180005 -0.52591997  0.20455     0.22532     0.19891     0.21863
 -0.14053001  0.026534    0.35482001 -0.27559    -0.14432999  0.14207999
 -0.23811001 -0.0045941  -0.14462    -0.10607    -0.23974     0.44398999
 -0.033788   -1.77400005 -0.97387999 -0.33886999  0.29912999 -0.21471
  2.93460011  0.47296    -0.069746   -0.42936999 -1.02279997 -1.10210001
 -1.14900005 -0.39353001 -0.46068001 -0.63748002 -0.38899001 -0.50265998
  0.92110002 -0.40483001 -0.19845     0.94019997 -0.59245998 -0.33818001
 -0.54872     0.41817999]


In [59]:
from keras.models import Sequential
from keras.layers import Embedding, Input
from keras.layers.merge import Concatenate
from keras.layers.core import Dense, Activation, Flatten
from keras.layers import Dropout, concatenate
from keras.layers.recurrent import LSTM
from keras.layers.wrappers import Bidirectional
from keras.layers.convolutional import Conv1D, MaxPooling1D
from keras.callbacks import ModelCheckpoint, EarlyStopping, TensorBoard
from keras import metrics
from keras.models import Model
import pickle

In [60]:
model = Sequential()
model.add(Embedding(input_dim=len(word_index) + 1,
                    output_dim=EMBEDDING_DIM,
                    weights=[embedding_matrix],
                    input_length=tweets_preprocessed.shape[1]))
model.add(LSTM(EMBEDDING_DIM, return_sequences = True, activation='relu'))
model.add(Dropout(.2))

model.add(LSTM(EMBEDDING_DIM, activation='relu'))
model.add(Dropout(.2))

model.add(Dense(32, activation='relu'))

model.add(Dense(1, activation='sigmoid'))

print(model.summary())

model.compile(loss = 'mean_squared_error', optimizer = 'RMSprop', metrics = [metrics.accuracy])

Model: "sequential_10"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_10 (Embedding)    (None, 118, 50)           34548050  
                                                                 
 lstm_18 (LSTM)              (None, 118, 50)           20200     
                                                                 
 dropout_16 (Dropout)        (None, 118, 50)           0         
                                                                 
 lstm_19 (LSTM)              (None, 50)                20200     
                                                                 
 dropout_17 (Dropout)        (None, 50)                0         
                                                                 
 dense_19 (Dense)            (None, 32)                1632      
                                                                 
 dense_20 (Dense)            (None, 1)               

In [ ]:
TEST_SIZE = 0.5
EPOCHS = 2
BATCH_SIZE = 32

model.fit(tweets_preprocessed, labels, 
          epochs = EPOCHS, 
          batch_size = BATCH_SIZE, 
          validation_split =TEST_SIZE)

Epoch 1/2
11082/25000 [============>.................] - ETA: 1:08:24 - loss: 0.0000e+00 - accuracy: 1.0000

In [ ]:
for x in range(800000, 800100):
  tweet_ex = np.array([tweets_preprocessed[x]])
  value = model.predict(tweet_ex)
  print(value[0])
  print (labels[x])